# Building you Deep Neural Network: Step by Step
We have previously trained a 2-layer network (with single hidden layer). This time we will build a deep neural network with as many layers as we want. 

**Notation**:
+ Superscript \[$l$\] denotes the quantity associated with the $l^{th}$ layer.
    + Eg: $a^{[L]}$ is the $L^{th}$ layer activation. $W^{[L]} \text{and } b^{[L]} \text{ are the } L^{th}$ layer parameters.
+ Superscript (i) denotes a quatity associated with the $i^{th}$ example.
    + Eg: $x^{(i)} \text{ is the } i^{th}$ training example.
+ Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    + Eg: $a^{[l]}_{i}$  denotes the $i^{th}$ entry of the $l^{th}$ layers activation.

## Setup

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
    
np.random.seed(1)    

## What we do?
+ Initialize the parameters for a two-layer network and for an L-layer neural network.
+ Implement forward propagation
    + Implement linear part of layer's forward propagation
    + Implement activation function (relu/sigmoid)
    + Combine previous two-steps into a new \[Linear -> Activation\] forward function.
    + Stack \[Linear -> RELU\] forward function L-1 time (for layers 1 through L-1) and add a \[Linear ->  Sigmoid\] at the end (of the final layer $L$). This gives us a new L_model_forward function.
+ Compute loss
+ Implement backward propagation
    + Implement linear part of layer's backward propagation step.
    + Implement the gradients of the activation function (relu_backward/sigmoid_backward)
    + Combine above two steps into a new \[Linear -> Activation\] backward function.
    + Stack \[Linear -> Relu\] backward L-1 times and add \[Linear ->  Sigmoid\] backward in a new L_model_backward function. 
+ Finally update the paramters.

**Note**: For every forward function, there is a corresponding backward function. That is why for every step of our forward module we will be storing some values in cache. The cached values are useful for computing gradients. In backpropagation module, we will then use cache to calculate the gradients. 

## 1. Initialization
Write helper function to initialize the paramters for the model. First function we will use it to initialize paramters for 2-layer model. The second one we will generalize this initialization process for $L$ layers.

### 1.1 2 - layer network
+ The model's structure is Linear -> RELU -> Linear -> RELU

In [2]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Arguments:
    ----------
    n_x: size of input layer
    n_h: size of hidden layer
    n_y: size of output layer
    
    Returns:
    --------
    parameters: dict, containing
                    W1: (n_h, n_x), weight matrix of hidden layer
                    b1: (n_h, 1), bias vector for hidden layer
                    W2: (n_y, n_h), weight matrix of output layer
                    b2: (n_y, 1), bias vector for output layer
    """
    np.random.seed(1)
    
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    paramters = {'W1': W1,
                 'b1': b1,
                 'W2': W2,
                 'b2': b2,}
    return parameters

### 1.2. L-layer network
+ Make sure dimensions match when initializing weigths and biases. Recall that $n^{[l]}$ is the number of units in the layer $l$. Thus for example, if the size of the input $X$ is (12288, 209) (with, m = 209 examples) then:

|Layer|Shape of W&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|Shape of b&nbsp;&nbsp;|Activation&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|Shape of Activation&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|-|--|--|--|--|
|Layer 1|$(n^{[1]}, 12288)$|$(n^{[1]}, 1)$|$Z^{[1]} = W^{[1]} X+ b^{[1]}$|$(n^{[1]}, 209)$|
|Layer 2|$(n^{[2]}, n^{[1]})$|$(n^{[2]}, 1)$|$Z^{[2]} = W^{[2]} A^{[1]}+ b^{[2]}$|$(n^{[2]}, 209)$|
|$\vdots$|$\vdots$|$\vdots$|$\vdots$|$\vdots$|
|Layer L-1|$(n^{[L-1]}, n^{[L-2]})$|$(n^{[L-1]}, 1)$|$Z^{[L-1]} = W^{[L-1]} A^{[L-2]}+ b^{[L-1]}$|$(n^{[L-1]}, 209)$|
|Layer L|$(n^{[L]}, n^{[L-1]})$|$(n^{[L]}, 1)$|$Z^{[L]} = W^{[L]} A^{[L-1]}+ b^{[L]}$|$(n^{[L]}, 209)$|

Remember that when we compute $W X + b$ in python, it carries out broadcasting. For example, if: 

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Then $WX + b$ will be:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

#### Implementing L-layer neural network
- The model's structure is *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*. I.e., it has $L-1$ layers using a ReLU activation function followed by an output layer with a sigmoid activation function.
- Use random initialization for the weight matrices. Use `np.random.rand(shape) * 0.01`.
- Use zeros initialization for the biases. Use `np.zeros(shape)`.
- We will store $n^{[l]}$, the number of units in different layers, in a variable `layer_dims`. For example, the `layer_dims` for the "Planar Data classification model" from last week would have been [2,4,1]: There were two inputs, one hidden layer with 4 hidden units, and an output layer with 1 output unit. Thus means `W1`'s shape was (4,2), `b1` was (4,1), `W2` was (1,4) and `b2` was (1,1). Now you will generalize this to $L$ layers! 
- Here is the implementation for $L=1$ (one layer neural network). It should inspire you to implement the general case (L-layer neural network).
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [3]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    ----------
    layer_dims: list, containing dimensions of each layer in the network
    
    Returns:
    --------
    parameters: dict, containing W1, b1, W2, b2, ...., WL, bL
                Wl = weight matrix of shape (layer_dims[l], layer_dims[l-1])
                bl = bias vector of shape (layer_dims[l], 1)
    """
    np.random.seed(3)
    
    parameters = {}
    for l in range(1, len(layer_dims)):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [4]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


## 2. Forward Propagation
### 2.1. Linear Forward
Now that you have initialized your parameters, you will do the forward propagation module. You will start by implementing some basic functions that you will use later when implementing the model. You will complete three functions in this order:

- LINEAR
- LINEAR -> ACTIVATION where ACTIVATION will be either ReLU or Sigmoid. 
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (whole model)

The linear forward module (vectorized over all the examples) computes the following equations:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

where $A^{[0]} = X$. 

**Exercise**: Build the linear part of forward propagation.

**Reminder**:
The mathematical representation of this unit is $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$. You may also find `np.dot()` useful. If your dimensions don't match, printing `W.shape` may help.

In [5]:
def linear_forward(A, W, b):
    """
    Implement linear part of the layer's forward propagation.
    
    Arguments:
    ----------
    A: activation from previous layer (or input data) - (size of previous layer, number of examples)
    W: weight matrix -- numpy array of shape - (size of current layer, size of previous layer)
    b: bias vector -- numpy array of shape - (size of current layer, 1)
    """
    Z = np.dot(W, A) + b
    
    assert Z.shape == (W.shape[0], A.shape[1])
    cache = (A, W, b)
    return Z, cache

In [6]:
def linear_forward_test_case():
    np.random.seed(1)
    A = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)
    return A, W, b

A, W, b = linear_forward_test_case()
Z, linear_cache = linear_forward(A, W, b)
print(f"Z = {Z}")

Z = [[ 3.26295337 -1.23429987]]


### 2.2. Linear-Activation Forward
We will use two activation functions
+ **Sigmoid: $\sigma(Z) = \sigma(W A +b) = \dfrac{1}{1 + e^{-(WA + b)}}$**. We have provided with `sigmoid` function. This function returns activation `A` and `Activation_cache` that contains `Z` (It's what we will feed in to the corresponding backward function. To use it you could just call:
```python
    A, activation_cache = sigmoid(Z)
```
+ **ReLU:** The mathematical formula for ReLu is, $A = RELU(Z) = max(0, Z)$. We have provided you with the `relu` function. This function returns two items: the activation value `A` and a `cache` that contains `Z` (it's what we will feed in to the corresponding backward function). To use it you could just call: 
```python
    A, activation_cache = relu(Z)
```

For more convenience, you are going to group two functions (Linear and Activation) into one function (LINEAR->ACTIVATION). Hence, you will implement a function that does the LINEAR forward step followed by an ACTIVATION forward step.

Implement the forward propagation of the LINEAR->ACTIVATION layer. Mathematical relation is: $A^{[l]} = g(Z^{[l]}) = g(W^{[l]} A^{[l-1]} + b^{[l]})$. where "g" can be "sigmoid" or "relu". Use linear_forward and correct the activation function.

In [7]:
def sigmoid(Z):
    """
    Compute sigmoid of the input function
    
    Arguments:
    ----------
    Z: numpy array of any shape
    
    Returns:
    --------
    A: output of sigmoid(z), same shape as Z 
    cache: returns Z as well, useful during backpropagation.
    """
    A = 1. / (1 + np.exp(-Z))
    cache = Z
    return A, cache 

def relu(Z):
    """
    Compute ReLU of the input function
    
    Arguments:
    ----------
    Z: numpy array of any shape
    
    Returns:
    --------
    A: output of relu(z), same shape as Z 
    cache: returns Z as well, useful during backpropagation.
    """
    A = np.maximum(0, Z)
    cache = Z
    return A, cache 
    
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- dict, containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert A.shape == (W.shape[0], A_prev.shape[1])    
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [8]:
def linear_activation_forward_test_case():
    np.random.seed(2)
    A_prev = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)
    return A_prev, W, b

A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Note**: In deep learning, the "\[LINEAR->ACTIVATION\]" computation is counted as a single layer in the neural network, not two layers.

For even more convenience when implementing the  𝐿 -layer Neural Net, you will need a function that replicates the previous one (`linear_activation_forward` with RELU)  𝐿−1  times, then follows that with one `linear_activation_forward` with SIGMOID.

+ Implement forward_propagation, `AL` will denote $A^{[L]} = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$. (This is sometimes also called `Yhat` i.e., $\hat{Y}$.

**Tips**:
- Use the functions you had previously written 
- Use a for loop to replicate [LINEAR->RELU] (L-1) times
- Don't forget to keep track of the caches in the "caches" list. To add a new value `c` to a `list`, you can use `list.append(c)`.

In [9]:
def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    ----------
    X: data, numpy array of shape (input size, number of examples)
    parameters: output of initialize_parameters_deep()
    
    Returns:
    --------
    AL: last post-activation value
    caches: list of caches containing-
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """
    caches = []
    A = X
    L = len(parameters) // 2  # no. of layers of neural network
    
    # Implement [Linear -> RELU] for L-1 layers
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, 
                                             parameters['W' + str(l)], 
                                             parameters['b' + str(l)], 
                                             activation='relu')
        caches.append(cache)
        
    # Implement final layer
    AL, cache = linear_activation_forward(A,
                                         parameters['W' + str(L)],
                                         parameters['b' + str(L)],
                                         activation='sigmoid')
    caches.append(cache)
    
    assert AL.shape == (parameters['W'+str(L)].shape[0], X.shape[1])   # (1, X.shape[1]), if final layer has only one neuron
    return AL, caches

In [10]:
def L_model_forward_test_case():
    np.random.seed(1)
    X = np.random.randn(4,2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return X, parameters

X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.17007265 0.2524272 ]]
Length of caches list = 2


Great! Now you have a full forward propagation that takes the input X and outputs a row vector $A^{[L]}$ containing your predictions. It also records all intermediate values in "caches". Using $A^{[L]}$, you can compute the cost of your predictions.

## 3. Cost function

Now you will implement forward and backward propagation. You need to compute the cost, because you want to check if your model is actually learning.

**Exercise**: Compute the cross-entropy cost $J$, using the following formula: $$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$

In [11]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    m = Y.shape[1]
    cost = (-1./m) * np.sum((Y * np.log(AL)) + ((1 - Y) * np.log(1 - AL)))
    cost = np.squeeze(cost)
    assert cost.shape == ()
    return cost

In [12]:
def compute_cost_test_case():
    Y = np.asarray([[1, 1, 1]])
    aL = np.array([[.8,.9,0.4]])
    return Y, aL

Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


## 4. Backward Propagation
Just like with forward propagation, you will implement helper functions for backpropagation. Remember that back propagation is used to calculate the gradient of the loss function with respect to the parameters.

![Backprop](images/backprop_kiank.png "Backprop")

Now, similar to forward propagation, build the backward propagation in three steps:

1. LINEAR backward
2. LINEAR -> ACTIVATION backward where ACTIVATION computes the derivative of either the ReLU or sigmoid activation
3. [LINEAR -> RELU]  ×  (L-1) -> LINEAR -> SIGMOID backward (whole model)

### 4.1. Linear Backward
For layer l, the linear part is: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$, followed by an activation function

Calculating, $dW^{[l]}, db^{[l]}, dA^{[l-1]}$:

derivative of loss function w.r.t 'activation' in layer '$l$' for '$i^{th}$' example is, 

\begin{equation}
\begin{aligned}
\dfrac{\partial \mathcal{L}} {\partial a^{[l](i)}} &= \dfrac{- \text{ }\partial (y^{(i)}\log(a^{[l](i)}) + (1 - y^{(i)})\log(1 - a^{[l](i)}))} {\partial a^{[l](i)}} \\
&= - \left(\dfrac{y^{(i)}} {a^{[l](i)}} - \dfrac{1 - y^{(i)}} {1 - a^{[l](i)}}\right) \\
&= - \dfrac{y^{(i)}} {a^{[l](i)}} + \dfrac{1 - y^{(i)}} {1 - a^{[l](i)}} 
\end{aligned}
\end{equation}

$$dA^{[l]} = \dfrac{\partial \mathcal{L}} {\partial A^{[l]}}$$

$$dZ^{[l]} = \dfrac{\partial \mathcal{L}} {{\partial Z^{[l]}}} = \dfrac{\partial \mathcal{L}} {\partial A^{[l]}} \text{ } \dfrac{\partial A^{[l]}} {\partial Z^{[l]}}$$

Suppose you have already calculated the derivative of $dZ^{[l]}$. You want to get ($dW^{[l]}, db^{[l]}, dA^{[l-1]}$).

<img src="images/linearback_kiank.png" alt="Linear back" height='350' width='350'>

\begin{equation}
\begin{aligned}
dW^{[l]} = \dfrac{\partial \mathcal{L}} {\partial W^{[l]}} &= \dfrac{\partial \mathcal{L}} {\partial A^{[l]}} \text{ } \dfrac{\partial A^{[l]}} {\partial Z^{[l]}} \text{ } \frac{\partial Z^{[l]}} {\partial W^{[l]}} \\
&= dZ^{[l]} \text{ } \dfrac{\partial (W^{[l]} A^{[l-1]} + b^{[l]})} {\partial W^{[l]}} \\
&= \frac{1}{m} dZ^{[l]} A^{[l-1]T}
\end{aligned}
\end{equation}

\begin{equation}
\begin{aligned}
db^{[l]} = \dfrac{\partial \mathcal{L}} {\partial b^{[l]}} &= \dfrac{\partial \mathcal{L}} {\partial A^{[l]}} \text{ } \dfrac{\partial A^{[l]}} {\partial Z^{[l]}} \text{ } \frac{\partial Z^{[l]}} {\partial b^{[l]}} \\
&= dZ^{[l]} \text{ } \dfrac{\partial (W^{[l]} A^{[l-1]} + b^{[l]})} {\partial b^{[l]}} \\
&= \frac{1}{m} \sum_{i=1}^{m} dZ^{[l](m)}
\end{aligned}
\end{equation}

\begin{equation}
\begin{aligned}
dA^{[l-1]} = \dfrac{\partial \mathcal{L}} {\partial A^{[l-1]}} &= \dfrac{\partial \mathcal{L}} {\partial A^{[l]}} \text{ } \dfrac{\partial A^{[l]}} {\partial Z^{[l]}} \text{ } \frac{\partial Z^{[l]}} {\partial A^{[l-1]}} \\
&= dZ^{[l]} \text{ } \dfrac{\partial (W^{[l]} A^{[l-1]} + b^{[l]})} {\partial A^{[l-1]}} \\
&= W^{[l]T} dZ^{[l]}
\end{aligned}
\end{equation}

In [13]:
def linear_backward(dZ, cache):
    """
    Implement linear portion of backward propagation for a single layer (layer l)
    
    Arguments:
    ----------
    dZ: gradient of the cost function with respect to linear output (of current layer l)
    cache: tuple, of values (A_prev, W, b) coming from the forward propagation in the current layer    
    
    dW: gradient of the cost function w.r.t 'W' (current layer l), same shape as W
    db: gradient of the cost function w.r.t 'b' (current layer l), same shape as b
    dA_prev: gradient of the cost function w.r.t  activation (of previous layer l-1), same shape as A_prev 
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = (1. / m) * np.dot(dZ, cache[0].T)
    db = (1. / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(cache[1].T, dZ)
    
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    assert (dA_prev.shape == (A_prev.shape))
    
    return dA_prev, dW, db

In [14]:
def linear_backward_test_case():
    """
    z, linear_cache = (np.array([[-0.8019545 ,  3.85763489]]), (np.array([[-1.02387576,  1.12397796],
       [-1.62328545,  0.64667545],
       [-1.74314104, -0.59664964]]), np.array([[ 0.74505627,  1.97611078, -1.24412333]]), np.array([[1]]))
    """
    np.random.seed(1)
    dZ = np.random.randn(1,2)
    A = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)
    linear_cache = (A, W, b)
    return dZ, linear_cache

# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


### 4.2. Linear-Activation Backward
Now we will create a function that merges the two helper functions: `linear_backward` and the backward step of activation `linear_activation_backward`.

Before implementing, write functions of activation backward propagation for **sigmoid** and **relu** 
+ **`sigmoid_backward`**: Implements backward propagation of SIGMOID unit. 
+ **`relu_backward`**: Implements backward propagation of RELU unit. 


If g(.) is the activation function, then `sigmoid_backward` and `relu_backward` compute

\begin{equation}
\begin{aligned}
dZ^{[l]} = \dfrac{\partial \mathcal{L}} {\partial Z^{[l]}} &= \dfrac{\partial \mathcal{L}} {\partial A^{[l]}} \text{ } \dfrac{\partial A^{[l]}} {\partial Z^{[l]}} \text{ } \\
&= dA^{[l]} \text{ . } 0 \text{ if} Z^{[l]} \leq 0 \text{ else } 1)  \text{, for RELU}\\
&= dA^{[l]} \text{ . } \sigma(Z^{[l]}) (1 - \sigma(Z^{[l]}))  \text{, for SIGMOID}
\end{aligned}
\end{equation}

$
\text{Here, } \\
\text{cross-entropy loss, } \mathcal{L} = -\dfrac{1}{m} \sum\limits_{i = 1}^{m} \left(y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)\right) = -\left(Y\log\left(A^{[l]}\right) + (1-Y)\log\left(1- A^{[l]}\right)\right) \\
dA^{[l]} = \dfrac{\partial \mathcal{L}} {\partial A^{[l]}} = -\left(\dfrac{Y} {A^{[l]}} - \dfrac{1 -Y} {1 - A^{[l]}}\right)
$

In [25]:
def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation of single SIGMOID unit.
    
    Arguments:
    ----------
    dA: post-activation gradient of any shape
    cache: 'Z', where we store for computing backward propagation efficiently
    
    Returns:
    --------
    dZ: gradient of the cost with respect to Z 
    """
    Z = cache
    
    s = 1./(1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    
    assert (dZ.shape == Z.shape)
    return dZ

def relu_backward(dA, cache):
    """
    Implement the backward propagation of a single RELU unit.
    
    Arguments:
    ----------
    dA: post-activation gradient of any shape
    cache: 'Z', where we store for computing backward propagation efficiently
    
    Returns:
    --------
    dZ: gradient of the cost with respect to Z
    """
    Z = cache
    
    dZ = np.array(dA, copy=True) # just converting dz to a correct object
    
    # when Z <= 0, then you should set dZ = 0 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    return dZ

In [26]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    ----------
    dA: post-activation gradient for current layer 'l'
    cache: tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently.
    activation: the activation to be used in this layer, stored as a text string, "sigmoid" or "relu"
    
    Returns:
    --------
    dA_prev: gradient of the cost function w.r.t activation (of previous layer, l-1), same shape as A_prev
    dW: gradient of the cost function w.r.t 'W' (current layer, l), same shape as W 
    db: gradient of the cost function w.r.t 'b' (current layer, l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [27]:
def linear_activation_backward_test_case():
    """
    aL, linear_activation_cache = (np.array([[ 3.1980455 ,  7.85763489]]), 
    ((np.array([[-1.02387576,  1.12397796], [-1.62328545,  0.64667545], [-1.74314104, -0.59664964]]), 
    np.array([[ 0.74505627,  1.97611078, -1.24412333]]), 5), np.array([[ 3.1980455 ,  7.85763489]])))
    """
    np.random.seed(2)
    dA = np.random.randn(1,2)
    A = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)
    Z = np.random.randn(1,2)
    linear_cache = (A, W, b)
    activation_cache = Z
    linear_activation_cache = (linear_cache, activation_cache)
    return dA, linear_activation_cache

AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


## 4.3. L-Model Backward
Now we will implement the backward function for the whole network. Recall that when you implemented the `L_model_forward` function, at each iteration, you stored a cache which contains (X, W, b, and Z). In the back propagation module, you will use those variables to compute the gradients. Therefore, in the `L_model_backward` function, you will iterate through all the hidden layers backward, starting from layer $L$ . On each step, you will use the cached values for layer $l$ to backpropagate through layer $l$. Figure below shows the backward pass

<img src="images/mn_backward.png" alt="Multi-layer NN backward propagation" height='500' width='500'>

**Initializing backpropagation**: To backpropagate through this network, we know that the output is, $A^{[L]} = \sigma(Z^{[L]})$. Our code thus needs to compute `dAL`$= \dfrac{\partial \mathcal{L}} {\partial A^{[L]}}$. To do so, use this formula derived above

```python
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # derivative of cost with respect to AL
```
We can then use this post-activation gradient `dAL` to keep going backward. As seen in Figure, we can now feed in `dAL` into the LINEAR->SIGMOID backward function we implemented (which will use the cached values stored by the L_model_forward function). After that, we will have to use a for loop to iterate through all the other layers using the LINEAR->RELU backward function. We should store each dA, dW, and db in the grads dictionary. To do so, use this formula :

$$grads["dW" + str(l)] = dW^{[l]}$$

For example, for $l$=3  this would store $dW^{[l]}$ in `grads["dW3"]`.

In [30]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL: probability vector, output of the forward propagation (L_model_forward())
    Y: true "label" vector (containing 0 if non-cat, 1 if cat)
    caches: list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this Y is the same shape as AL
    
    # Initializing backpropagation
    dAL = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation="relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads

In [31]:
def L_model_backward_test_case():
    """
    X = np.random.rand(3,2)
    Y = np.array([[1, 1]])
    parameters = {'W1': np.array([[ 1.78862847,  0.43650985,  0.09649747]]), 'b1': np.array([[ 0.]])}

    aL, caches = (np.array([[ 0.60298372,  0.87182628]]), [((np.array([[ 0.20445225,  0.87811744],
           [ 0.02738759,  0.67046751],
           [ 0.4173048 ,  0.55868983]]),
    np.array([[ 1.78862847,  0.43650985,  0.09649747]]),
    np.array([[ 0.]])),
   np.array([[ 0.41791293,  1.91720367]]))])
   """
    np.random.seed(3)
    AL = np.random.randn(1, 2)
    Y = np.array([[1, 0]])

    A1 = np.random.randn(4,2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    Z1 = np.random.randn(3,2)
    linear_cache_activation_1 = ((A1, W1, b1), Z1)

    A2 = np.random.randn(3,2)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)
    Z2 = np.random.randn(1,2)
    linear_cache_activation_2 = ( (A2, W2, b2), Z2)

    caches = (linear_cache_activation_1, linear_cache_activation_2)
    return AL, Y, caches

AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


## 5. Update Parameters

In this section you will update the parameters of the model, using gradient descent: 

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} $$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} $$

where $\alpha$ is the learning rate. After computing the updated parameters, store them in the parameters dictionary. 

**Exercise**: Implement `update_parameters()` to update your parameters using gradient descent.

**Instructions**:
Update parameters using gradient descent on every $W^{[l]}$ and $b^{[l]}$ for $l = 1, 2, ..., L$. 

In [33]:
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    L = len(parameters) // 2 # number of layers in the neural network
    
    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [34]:
def update_parameters_test_case():
    """
    parameters = {'W1': np.array([[ 1.78862847,  0.43650985,  0.09649747],
        [-1.8634927 , -0.2773882 , -0.35475898],
        [-0.08274148, -0.62700068, -0.04381817],
        [-0.47721803, -1.31386475,  0.88462238]]),
 'W2': np.array([[ 0.88131804,  1.70957306,  0.05003364, -0.40467741],
        [-0.54535995, -1.54647732,  0.98236743, -1.10106763],
        [-1.18504653, -0.2056499 ,  1.48614836,  0.23671627]]),
 'W3': np.array([[-1.02378514, -0.7129932 ,  0.62524497],
        [-0.16051336, -0.76883635, -0.23003072]]),
 'b1': np.array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
 'b2': np.array([[ 0.],
        [ 0.],
        [ 0.]]),
 'b3': np.array([[ 0.],
        [ 0.]])}
    grads = {'dW1': np.array([[ 0.63070583,  0.66482653,  0.18308507],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]),
 'dW2': np.array([[ 1.62934255,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]),
 'dW3': np.array([[-1.40260776,  0.        ,  0.        ]]),
 'da1': np.array([[ 0.70760786,  0.65063504],
        [ 0.17268975,  0.15878569],
        [ 0.03817582,  0.03510211]]),
 'da2': np.array([[ 0.39561478,  0.36376198],
        [ 0.7674101 ,  0.70562233],
        [ 0.0224596 ,  0.02065127],
        [-0.18165561, -0.16702967]]),
 'da3': np.array([[ 0.44888991,  0.41274769],
        [ 0.31261975,  0.28744927],
        [-0.27414557, -0.25207283]]),
 'db1': 0.75937676204411464,
 'db2': 0.86163759922811056,
 'db3': -0.84161956022334572}
    """
    np.random.seed(2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    np.random.seed(3)
    dW1 = np.random.randn(3,4)
    db1 = np.random.randn(3,1)
    dW2 = np.random.randn(1,3)
    db2 = np.random.randn(1,1)
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return parameters, grads

parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


## 6. Conclusion
We have implemented all the functions whihch are required for building a Deep Neural Network

In the next assignment we will put all together and build two models:
+ A two-layer neural network
+ An L-layer neural network 